In [ ]:
import pandas as pd
import os

# *Sorghum-FGVC9*

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26245/logos/header.png?t=2021-04-07-18-10-13)



# *Import libraries for Data Preprocessing*

In [ ]:


import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from termcolor import colored
from PIL import Image



# *Define data path file*

In [ ]:


TRAIN_IMAGE_DIR = '../input/sorghum-id-fgvc-9/train_images'
TEST_IMAGE_DIR = '../input/sorghum-id-fgvc-9/test'

TRAIN_IMAGE_SAVE_DIR = '/kaggle/working/resized_train_images'
TEST_IMAGE_SAVE_DIR = '/kaggle/working/resized_test_images'



# *Count Images in folder train and test*

In [ ]:


train_images = os.listdir(TRAIN_IMAGE_DIR)
test_images = os.listdir(TEST_IMAGE_DIR)

n_train_images = len(train_images)
n_test_images = len(test_images)

print('# of training images = {}'.format(n_train_images))
print('# of testing images = {}'.format(n_test_images))



# *Create function for resize images with 3 channel(RGB) and image size*

In [ ]:
def resize_images(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    return img


# *The loop for resizing images in folder and put it into the new folder*

In [ ]:


print(colored('Processing training images...', 'red'))
if not os.path.exists(TRAIN_IMAGE_SAVE_DIR):
    os.makedirs(TRAIN_IMAGE_SAVE_DIR)
    print('Training images saving directory created')
else:
    print('Training images saving directory exists')
    
for i, train_image in enumerate(train_images):
    if (i+1) % 1000 == 0:
        print('{}/{} images processed'.format(i+1, n_train_images))
    resized_img = resize_images(os.path.join(TRAIN_IMAGE_DIR, train_image))
    tf.keras.utils.save_img(os.path.join(TRAIN_IMAGE_SAVE_DIR, train_image), resized_img)



# *Zipfile if you want to download it and use like custom datasets*

In [ ]:
!zip -r resize_train.zip ./resized_train_images

# *Create CSV file for get path and label from folder datasets*

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
DATA_DIR  = './'
TRAIN_DIR = DATA_DIR + 'resized_train_images/'
# TEST_DIR  = DATA_DIR + 'resized_test_images/'



label_encoder = LabelEncoder()

# Load Train Data
train_df = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv')
train_df['Id'] = train_df['image'].apply(lambda x: f'{TRAIN_DIR}{x}')
# Summary
print(f'train_df: {train_df.shape}')
train_df.head()


# *Count classes from column cultivars in csv file*

In [ ]:
unique_cultivars = list(train_df.cultivar.unique())
num_classes = len(unique_cultivars)
num_classes

In [ ]:
train_df['Id']

In [ ]:
train_df['cultivar']

In [ ]:
train_df.to_csv('train.csv')

In [ ]:
data = pd.read_csv('./train.csv')
# data['file_path'] = data['file_path'].astype('str')



In [ ]:
data


# *Import libraries prepare for data augmentation and training loop processing*

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201, EfficientNetB3, InceptionV3, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import os
import pandas as pd

# *Default epochs, batch_size and img_size*

In [ ]:
EPOCHS = 30
BATCH_SIZE = 256
IMG_SIZE = 224

In [ ]:
cultivars = data['cultivar'].to_numpy()
cultivars = cultivars
cultivars

# *If you want to use data aug from keras you have to change dtype of column cultivar from object to string type*

In [ ]:
data['cultivar'] = data['cultivar'].astype('str')
# train_df['Id'] = train_df['Id'].astype('string')

In [ ]:
data['cultivar']

# *Data augmentation*

![](https://www.researchgate.net/publication/347221279/figure/fig1/AS:1023619987673089@1621061431785/Image-data-augmentation-technique-for-increasing-the-size-of-an-image-to-train-a-deep.png)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    preprocessing_function=preprocess_input,
    validation_split=0.2)

train_generator=datagen.flow_from_dataframe(
dataframe=data,
# directory="./train/",
x_col="Id",
y_col="cultivar",
subset="training",
batch_size=BATCH_SIZE,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))


valid_generator=datagen.flow_from_dataframe(
dataframe=data,
# directory="./train/",
x_col="Id",
y_col="cultivar",
subset="validation",
batch_size=BATCH_SIZE,
shuffle=True,
class_mode="categorical",
target_size=(IMG_SIZE,IMG_SIZE))


test_datagen = ImageDataGenerator()

# *Here, this is transfer learning using pretrained model InceptionV3, so the question is asked here Why we use InceptionV3, so until now we have many pretrained model and we can't use them indiscriminately, we have to know how we can extract the features of image in the best way with pretrained model, so in here I use InceptionV3 because, our image has many detail and we need this pretrained to take features and of course It's very deep*

![](https://res.cloudinary.com/practicaldev/image/fetch/s--eKV0swu_--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/https://dev-to-uploads.s3.amazonaws.com/i/2oyycxxs02jmcghplwc4.png)

![](https://camo.githubusercontent.com/8b243e646673dd9234f39cf8bdd5da1c6f051fd9/68747470733a2f2f7777772e50657465724d6f7373416d6c416c6c52657365617263682e636f6d2f6d656469612f696d616765732f7265706f7369746f726965732f5472616e736665722d4c6561726e696e672e6a7067)

In [ ]:
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

for layer in base_model.layers:
    layer.trainable = True

head_model = base_model.output
head_model = GlobalAveragePooling2D()(head_model)
head_model = Flatten()(head_model)
# head_model = Dense(128, activation="relu")(head_model)
# head_model = Dropout(0.4)(head_model)
# head_model = Dense(64, activation="relu")(head_model)
head_model = Dense(100, activation="softmax")(head_model)
model = Model(inputs=base_model.input, outputs=head_model)
model.summary()

# opt = Adam(learning_rate=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# *Define checkpoint and early stopping for have the best model*

In [ ]:

cp_callback = ModelCheckpoint(filepath='fgvc9_model_4.h5',
                              monitor='val_accuracy',
                              save_freq='epoch', verbose=1, period=1,
                              save_best_only=True, save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_accuracy',
                               verbose=1, patience=5)



# *Reduce learning rate on each epoch*

In [ ]:
from datetime import datetime, timedelta
import tensorflow as tf
import matplotlib.pyplot as plt
start_time = datetime.now()
print('Time now is', start_time)
end_training_by_tdelta = timedelta(seconds=8400)
this_run_file_prefix = start_time.strftime('%Y%m%d_%H%M_')


# EPOCHS = 25
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE



# Learning Rate Schedule for Fine Tuning #
# Learning rate schedule for TPU, GPU and CPU.
# Using an LR ramp up because fine-tuning a pre-trained model.
# Starting with a high LR would break the pre-trained weights.

LR_START = 0.00001
LR_MAX = 0.00005 * 8
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8
num_epochs = 50
def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = LR_START + (epoch * (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS)
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
rng = [i for i in range(num_epochs)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))



# *Model Fitting*

In [ ]:
history = model.fit(
                train_generator, 
                validation_data=valid_generator,
                epochs=num_epochs,
                steps_per_epoch=train_generator.samples // train_generator.batch_size,
#                 validation_steps= (3712 / 256)*2,
                callbacks=[cp_callback, early_stopping, lr_callback])



In [ ]:
model.save('fgvc_9_best_model.h5')

In [ ]:
model = load_model('../input/model-trained/fgvc_9_best_model.h5')

In [ ]:
!nvidia-smi

In [ ]:
cultivars = data['cultivar'].to_numpy()
cultivars = cultivars
cultivars

In [ ]:
class_names = np.unique(cultivars)
num_classes = len(class_names)
num_classes

# *Create Test Dataframe prepare for Prediction*

In [ ]:
test_datagen = ImageDataGenerator()
test_filenames = os.listdir("../input/sorghum-id-fgvc-9/test")
test_df = pd.DataFrame({
    'filename': test_filenames
})
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    "../input/sorghum-id-fgvc-9/test",
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)


# *Prediction*

In [ ]:
# test_ds = test_ds.batch(BATCH_SIZE)
preds = model.predict(test_generator)
preds = np.argmax(preds, axis=1)
preds = [class_names[i] for i in preds]
preds

# *Submission*

In [ ]:
test_df['cultivar'] = preds
test_df.to_csv('submission.csv', index=False)

In [ ]:
test_df